In [1]:
from abc import ABC, abstractmethod
from collections import deque

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

In [2]:
def distance(a, b):
    xA, yA = a
    xB, yB = b
    return np.hypot((xA - xB), (yA - yB))

In [ ]:
# show map

In [3]:
class Problem(ABC):
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal
        
    @abstractmethod
    def actions(self, state):
        pass
    
    @abstractmethod
    def result(self, state, action):
        pass
    
    def goal_test(self, state):
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        else:
            return state == self.goal

In [4]:
class GraphProblem(Problem):
    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal)
        self.graph = graph
        
    def actions(self, A):
        return list(self.graph.get(A).keys())
    
    def result(self, state, action):
        return action


# Graph Class

In [5]:
class Graph:
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
            
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect(b, a, dist)
    
    def connect(self, A, B, distance):
        self.graph_dict.setdefault(A, {})[B] = distance
    
    
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        return links if b is None else links.get(b)
    
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        return list()
    

# undirected graph function
def UndirectedGraph(graph_dict=None):
    return Graph(graph_dict=graph_dict, directed=False)
        

# Node Class

In [6]:
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
    
    def __repr__(self):
        return f"<Node {self.state}>"
    
    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]
    
    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action)
        return next_node
    
    def solution(self):
        return [node.action for node in self.path()[1:]]
    
    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    

In [7]:
romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

In [8]:
romania_map.graph_dict

{'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
 'Bucharest': {'Urziceni': 85, 'Pitesti': 101, 'Giurgiu': 90, 'Fagaras': 211},
 'Craiova': {'Drobeta': 120, 'Rimnicu': 146, 'Pitesti': 138},
 'Drobeta': {'Mehadia': 75, 'Craiova': 120},
 'Eforie': {'Hirsova': 86},
 'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
 'Hirsova': {'Urziceni': 98, 'Eforie': 86},
 'Iasi': {'Vaslui': 92, 'Neamt': 87},
 'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
 'Oradea': {'Zerind': 71, 'Sibiu': 151},
 'Pitesti': {'Rimnicu': 97, 'Bucharest': 101, 'Craiova': 138},
 'Rimnicu': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
 'Urziceni': {'Vaslui': 142, 'Bucharest': 85, 'Hirsova': 98},
 'Zerind': {'Arad': 75, 'Oradea': 71},
 'Sibiu': {'Arad': 140, 'Fagaras': 99, 'Oradea': 151, 'Rimnicu': 80},
 'Timisoara': {'Arad': 118, 'Lugoj': 111},
 'Giurgiu': {'Bucharest': 90},
 'Mehadia': {'Drobeta': 75, 'Lugoj': 70},
 'Vaslui': {'Iasi': 92, 'Urziceni': 142},
 'Neamt': {'Iasi': 87}}

In [20]:
romania_problem = GraphProblem('Bucharest', 'Arad', romania_map)

In [30]:
def bfs(problem):    
    
    iterations = 0
    all_node_colors = []
    node_colors = {k: 'white' for k in problem.graph.nodes()}
    node = Node(problem.initial)
      
    if problem.goal_test(node.state):
        return node
    
    frontier = deque([node])
    explored = set()
    path = []
    
    while frontier:
        node = frontier.popleft()        
        explored.add(node.state)        
        for child in (expand := node.expand(problem)):
            
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    print(child)
                    return child.path()
                frontier.append(child)
    
    return None

In [32]:
start = 'Arad'
end = 'Bucharest'
graph = romania_map

romania_problem = GraphProblem(start, end, graph)
path = bfs(romania_problem)

[print(p.state) for p in path]

<Node Bucharest>
Arad
Sibiu
Fagaras
Bucharest


[None, None, None, None]

In [3]:
a = dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)
)
a

{'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
 'Bucharest': {'Urziceni': 85, 'Pitesti': 101, 'Giurgiu': 90, 'Fagaras': 211},
 'Craiova': {'Drobeta': 120, 'Rimnicu': 146, 'Pitesti': 138},
 'Drobeta': {'Mehadia': 75},
 'Eforie': {'Hirsova': 86},
 'Fagaras': {'Sibiu': 99},
 'Hirsova': {'Urziceni': 98},
 'Iasi': {'Vaslui': 92, 'Neamt': 87},
 'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
 'Oradea': {'Zerind': 71, 'Sibiu': 151},
 'Pitesti': {'Rimnicu': 97},
 'Rimnicu': {'Sibiu': 80},
 'Urziceni': {'Vaslui': 142}}

In [6]:
s1 = set([k for k in a.keys()])
s1

{'Arad',
 'Bucharest',
 'Craiova',
 'Drobeta',
 'Eforie',
 'Fagaras',
 'Hirsova',
 'Iasi',
 'Lugoj',
 'Oradea',
 'Pitesti',
 'Rimnicu',
 'Urziceni'}

In [ ]:
s2 = set([k ])